In [1]:
#train an CNN model for the accent detector
#for low tech demo: onlhy destringuish us accents with others 

#import models to be used
import numpy as np
import pandas as pd
import librosa
import multiprocessing
import sys


from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from keras import utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import MaxPooling2D, Conv2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, TensorBoard

Using TensorFlow backend.


In [2]:
DEBUG = True
SILENCE_THRESHOLD = .01
RATE = 24000
N_MFCC = 13
COL_SIZE = 30
EPOCHS = 10 #35#250

In [82]:
#get audio data and information from speech accent archive using Webscraping
import os # To interact with operating system and format file name
import shutil # To copy file object from python to disk
import requests
import bs4 as bs 
from collections import Counter
import re
import urllib.request
import sys
from pydub import AudioSegment
#source = requests.get("http://accent.gmu.edu/browse_language.php")
#soup = bs.BeautifulSoup(source.content, features='html.parser') 
#print(soup.prettify())


/Users/yidanyuan/anaconda3/lib/python3.7/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [86]:
"""specify desired languages; 
for now (check point #1): only english and mandarin are used;
eventually/ideally, all languages will be used"""
languages = ["mandarin","english"]
URL = 'http://accent.gmu.edu/'
mp3_url = 'http://chnm.gmu.edu/accent/soundtracks/{}.mp3'


In [87]:
#get data and bio information from the url
def create_df(languages = languages, url = URL):
    import bs4 as bs 
    search_url = [url+'browse_language.php?function=find&language={}'.format(l) for l in languages]
    htmls = []
    for url in search_url:
        #print('downloading from {}'.format(url))
        htmls.append(requests.get(url).text)
    #print(htmls)
    
    bss = [bs.BeautifulSoup(html,'html.parser') for html in htmls]
    #print(bss)
    people = []
    counter = 0
    for b in bss:
        
        for p in b.find_all('p'):
            counter += 1
            if p.a:
                text = p.text.replace(' ','').split(',')
                address = 'http://accent.gmu.edu/'+p.a['href']
                speakerid = address.split('speakerid=')[1]
                #print(speakerid)
                s = bs.BeautifulSoup(requests.get(address).text,'html.parser')
                each_bio = [li.text for li in s.find('ul','bio').find_all('li')]
                native_language = each_bio[1].split(': ')[1]
                eng_residence = each_bio[6].split(': ')[1]
                if native_language.find("english") != -1 and eng_residence.find('usa') != -1:
                    if_us_native = 1
                else:
                    if_us_native = 0
                x = [speakerid,address, text[0], text[1],native_language, if_us_native]
                #print(counter)
                #print(x)
                
                people.append(x)
                                
    df = pd.DataFrame(people, columns=['speakerid','href','language_num','gender',\
                                       'native_language','if_us_native'])
    
    return df

In [80]:
df = create_df()

256
257
258
259
260
261
262
430
451
491
607
623
685
719
746
750
796
872
916
931
984
1019
1044
1089
1125
1141
1154
1447
1490
1491
1519
1527
1541
1578
1579
1580
1581
1582
1583
1614
1615
1616
1617
1630
1631
1636
1644
1645
1736
1788
1792
1793
1817
1900
1909
1910
1990
2000
2019
2109
2116
2117
2119
2129
2173
2252
2274
2281
2282
2291
2294
2306
2319
2323
2332
2348
2355
2360
2365
2413
2414
2418
2419
2420
2421
2423
2435
2447
2448
2461
2463
2469
2474
2477
2483
2494
2523
2524
2529
2530
2531
2532
2538
2539
2540
2542
2554
2557
2559
2560
2562
2564
2566
2599
2675
2677
2678
2679
2699
2700
2708
2711
2714
2716
2719
2726
2748
2772
2773
2798
2836
2837
61
77
88
99
110
121
132
143
154
62
68
69
70
71
72
73
74
75
76
78
79
80
81
82
83
84
85
86
87
89
90
91
92
93
94
95
96
97
98
100
101
102
103
104
105
106
107
108
109
111
112
113
114
115
116
117
118
119
120
122
123
124
125
126
127
128
129
130
131
133
134
135
136
137
138
139
140
141
142
144
145
146
147
148
149
150
151
152
153
155
156
157
158
159
160
161
162
163
164

In [107]:
#some helper code scratch
#df.groupby('if_us_native').count()
"""source = bs.BeautifulSoup(requests.get(df['href'][len(df['href'])-1]).text,'html.parser')
each_bio = [li.text for li in source.find('ul','bio').find_all('li')]
each_bio"""
path = os.getcwd()  
print ("The current working directory is %s" % path)

The current working directory is /Users/yidanyuan/Documents/Academic/spring2019/ind135


In [114]:
df.groupby('if_us_native').count()


,speakerid,href,language_num,sex,native_language
if_us_native,,,,,
0,301,301,301,301,301
1,457,457,457,457,457


In [115]:
def get_audio(df,destination_folder,url = mp3_url):
    
    if not os.path.exists(destination_folder):
        print('creating audio destination folder'.format(destination_folder))
        os.makedirs(os.getcwd() + "/" + destination_folder)
    else:
        print('audio destination folder already exists.')
            
    counter = 0
    for lang_num in df['language_num']:
        if not os.path.exists(destination_folder +'{}.wav'.format(lang_num)):
            print('downloading {}'.format(lang_num))
            (filename, headers) = urllib.request.urlretrieve(url.format(lang_num))
            sound = AudioSegment.from_mp3(filename)
            sound.export(os.getcwd() +"/"+ destination_folder + "{}.wav".format(lang_num),\
                         format="wav")
            counter += 1

    return counter

In [116]:
get_audio(df, 'wav')

creating audio destination folder
downloading mandarin1
downloading mandarin2
downloading mandarin3
downloading mandarin4
downloading mandarin5
downloading mandarin6
downloading mandarin7
downloading mandarin8
downloading mandarin9
downloading mandarin10
downloading mandarin11
downloading mandarin12
downloading mandarin13
downloading mandarin14
downloading mandarin15
downloading mandarin16
downloading mandarin17
downloading mandarin18
downloading mandarin19
downloading mandarin20
downloading mandarin21
downloading mandarin22
downloading mandarin23
downloading mandarin24
downloading mandarin25
downloading mandarin26
downloading mandarin27
downloading mandarin28
downloading mandarin29
downloading mandarin30
downloading mandarin31
downloading mandarin32
downloading mandarin33
downloading mandarin34
downloading mandarin35
downloading mandarin36
downloading mandarin37
downloading mandarin38
downloading mandarin39
downloading mandarin40
downloading mandarin41
downloading mandarin42
downloadi

downloading english228
downloading english229
downloading english230
downloading english231
downloading english232
downloading english233
downloading english234
downloading english235
downloading english236
downloading english237
downloading english238
downloading english239
downloading english240
downloading english241
downloading english242
downloading english243
downloading english244
downloading english245
downloading english246
downloading english247
downloading english248
downloading english249
downloading english250
downloading english251
downloading english252
downloading english253
downloading english254
downloading english255
downloading english256
downloading english257
downloading english258
downloading english259
downloading english260
downloading english261
downloading english262
downloading english263
downloading english264
downloading english265
downloading english266
downloading english267
downloading english268
downloading english269
downloading english270
downloading

downloading english585
downloading english586
downloading english587
downloading english588
downloading english589
downloading english590
downloading english591
downloading english592
downloading english593
downloading english594
downloading english595
downloading english596
downloading english597
downloading english598
downloading english599
downloading english600
downloading english601
downloading english602
downloading english603
downloading english604
downloading english605
downloading english606
downloading english607
downloading english608
downloading english609
downloading english610
downloading english611
downloading english612
downloading english613
downloading english614
downloading english615
downloading english616
downloading english617
downloading english618
downloading english619
downloading english620
downloading english621
downloading english622
downloading english623
downloading english624
downloading english625
downloading english626


758